# Chapter 7: Measuring Uncertainty With the Bootstrap

In [1]:
##### Data and libraries

# Common libraries
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import seaborn as sns

# Chapter-specific libraries
import statsmodels.stats.outliers_influence as st_inf # For Cook's distance
import matplotlib.pyplot as plt
import statsmodels.api as sm # For QQ plot

### Generating the data

times = [2,2,3,5,6,9,10,47,61,413]
experience = [11,17,18,1,10,4,6,3,8,0]

data_df = pd.DataFrame(
    {'times': times, 
    'experience': experience})

## Intro to the Bootstrap: Polling Oneself Up

### The Business Problem: Small Data with an Outlier

In [2]:
# Building linear model
lin_mod = ols("times~1", data=data_df).fit()
print(lin_mod.summary())

est = lin_mod.params['Intercept']
se = lin_mod.bse['Intercept']

#Building normal confidence interval
LL = est-1.96*se #Lower limit
UL = est+1.96*se #Upper limit
print("LL = ", LL)
print("UL = ",UL)

                            OLS Regression Results                            
Dep. Variable:                  times   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 11 May 2023   Prob (F-statistic):                nan
Time:                        14:01:41   Log-Likelihood:                -62.120
No. Observations:                  10   AIC:                             126.2
Df Residuals:                       9   BIC:                             126.5
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     55.8000     40.225      1.387      0.1

C:\Users\Florent\Anaconda3\lib\site-packages\scipy\stats\stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  "anyway, n=%i" % int(n))


### Bootstrap Confidence Interval for the Sample Mean

In [4]:
#Building bootstrap CI
res_boot_sim = [] 
B = 2000
N = len(data_df)
for i in range(B):
    boot_df = data_df.sample(N, replace = True)
    M = np.mean(boot_df.times)
    res_boot_sim.append(M)
    
LL_b = np.quantile(res_boot_sim, 0.025)  
UL_b = np.quantile(res_boot_sim, 0.975)

# Note that the following values are random and will vary across draws
print("LL_b = ", LL_b)
print("UL_b = ",UL_b)

LL_b =  5.9
UL_b =  137.9025


### Bootstrap Confidence Intervals for Ad Hoc Statistics

In [ ]:
#### Bootstrap for time promise
promise_lst = []
B = 2000
N = len(data_df)
for i in range(B):
    boot_df = data_df.sample(N, replace = True)
    above180 =  len(boot_df[boot_df.times >= 180]) / N
    promise_lst.append(above180)
LL_b = np.quantile(promise_lst, 0.025)  
UL_b = np.quantile(promise_lst, 0.975)  
print("LL_b = ", LL_b)
print("UL_b = ",UL_b)

## The Bootstrap for Regression Analysis

In [ ]:
print(ols("times~experience", data=data_df).fit().summary())

In [ ]:
reg_lst = []
B = 4000
N = len(data_df)
for i in range(B):
    boot_df = data_df.sample(N, replace = True)
    lin_mod = ols("times~experience", data=boot_df).fit()
    coeff = lin_mod.params['experience']
    reg_lst.append(coeff)
LL_b = np.quantile(reg_lst, 0.025)  
UL_b = np.quantile(reg_lst, 0.975) 

sns.displot(reg_lst) 

In [ ]:
pval = 2 * sum(1 for x in reg_lst if x > 0) / B
print(pval)

## When to use the Bootstrap

In [ ]:
lin_mod = ols("times~experience", data=data_df).fit()
print(lin_mod.summary())

In [ ]:
#Extract Cook's distance for influential points
CD = st_inf.OLSInfluence(lin_mod).summary_frame()['cooks_d']
CD[CD > 1]

In [ ]:
#Density plot of residuals
res_df = lin_mod.resid
sns.kdeplot(res_df)

In [ ]:
#QQ plot of residuals
fig = sm.qqplot(res_df, line='s')
plt.show()

## Optimizing the Bootstrap in R and Python

In [ ]:
#Creating unique numpy array for sampling
data_ar = data_df.to_numpy()
rng = np.random.default_rng()

np_lst = []
for i in range(B): 
    
    #Extracting the relevant columns from array
    boot_ar = rng.choice(data_ar, size=N, replace=True)
    X = boot_ar[:,1]
    X = np.c_[X, np.ones(N)]
    Y = boot_ar[:,0]
    
    ### LSTQ implementation
    np_lst.append(np.linalg.lstsq(X, Y, rcond=-1)[0][0])

#Plotting histogram
sns.displot(np_lst)

LL_b_np = np.quantile(np_lst, 0.025)  
UL_b_np = np.quantile(np_lst, 0.975) 